In [10]:
# Asignar directorios de trabajo a "./someweird"
import os
# os.chdir("./someweird")

DATA_DIR = "./data/20180702_1145_PREP/"
FILE_DATA_PATH = DATA_DIR + "QRO_DIP_LOC_2018.csv"

#
# Cargar archivos de .data/20180702_1145_PREP/QRO_DIP_LOC_2018.csv

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d

# table start in A6

data = pd.read_csv(FILE_DATA_PATH, header=4, dtype=str)
data



# Realizar analisis de datos

,CLAVE_CASILLA,CLAVE_ACTA,ID_ESTADO,ESTADO,ID_DISTRITO_LOCAL,DISTRITO_LOCAL,SECCION,ID_CASILLA,TIPO_CASILLA,EXT_CONTIGUA,...,OBSERVACIONES,CONTABILIZADA,MECANISMOS_TRASLADO,SHA,FECHA_HORA_ACOPIO,FECHA_HORA_CAPTURA,FECHA_HORA_VERIFICACION,ORIGEN,DIGITALIZACION,TIPO_DOCUMENTO
0,220277B0100,220277B0100,22,QUERETARO,1,QUERETARO 01,0277,961,B,00,...,NaN,1,D,77B4E74C39AA83026388F7148239AF76816E8B6D29CC03...,02/07/2018 01:18:43,02/07/2018 02:18:53,02/07/2018 04:25:39,CATD,ESCANER,ACTA PREP
1,220277C0100,220277C0100,22,QUERETARO,1,QUERETARO 01,0277,962,C,00,...,NaN,1,D,C055A6A27C574F57BE0185476BC19756D0290CB26C9FB6...,02/07/2018 01:17:20,02/07/2018 04:40:41,02/07/2018 05:41:19,CATD,ESCANER,ACTA PREP
2,220277C0200,220277C0200,22,QUERETARO,1,QUERETARO 01,0277,963,C,00,...,NaN,1,D,83A57141BC2CB59F7A4D43069AF9DBBA5B13B61320FC59...,02/07/2018 00:42:16,02/07/2018 01:03:58,02/07/2018 01:15:12,CATD,ESCANER,ACTA PREP
3,220277C0300,220277C0300,22,QUERETARO,1,QUERETARO 01,0277,964,C,00,...,NaN,1,D,AB96359CA2A806F9B8A96D9666FC3D74571BA564F32DA1...,02/07/2018 01:18:11,02/07/2018 02:40:52,02/07/2018 05:05:53,CATD,ESCANER,ACTA PREP
4,220277C0400,220277C0400,22,QUERETARO,1,QUERETARO 01,0277,965,C,00,...,NaN,1,D,4D527753D1E7D940AEA1CE9FF61E66A3237D5255359759...,02/07/2018 00:42:51,02/07/2018 01:09:11,02/07/2018 01:37:26,CATD,ESCANER,ACTA PREP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2643,'220560E0100,'220560E0100,22,QUERETARO,15,JALPAN DE SERRA,0560,1866,E,00,...,NaN,1,D,3D4784EAF0E8D2B440B5C1E69CBABAB6529198038D9AEF...,02/07/2018 04:01:00,02/07/2018 04:09:51,02/07/2018 04:13:08,CATD,ESCANER,ACTA PREP
2644,'220560E0200,'220560E0200,22,QUERETARO,15,JALPAN DE SERRA,0560,1867,E,00,...,NaN,1,D,132AB7B5B93DB99D91A4730A20E60B93D08C5A867296E0...,02/07/2018 01:24:12,02/07/2018 01:41:11,02/07/2018 01:50:03,CATD,ESCANER,ACTA PREP
2645,220561B0100,220561B0100,22,QUERETARO,15,JALPAN DE SERRA,0561,1868,B,00,...,NaN,1,D,C6FFB9FC04C0ED64E2932E4880A85007BD4BD02B3E7C6C...,02/07/2018 00:52:07,02/07/2018 01:03:10,02/07/2018 01:08:31,CATD,ESCANER,ACTA PREP
2646,'220561E0100,'220561E0100,22,QUERETARO,15,JALPAN DE SERRA,0561,1869,E,00,...,NaN,1,D,5DA938A7C3E4466246F732E7FCC1BA4D9F084FF3D1764C...,02/07/2018 00:50:46,02/07/2018 01:04:55,02/07/2018 01:06:02,CATD,ESCANER,ACTA PREP


In [11]:
def resume_column(columna, title=None):
	columna_frecuencia = columna.value_counts().sort_values(ascending=False)
	print(f'Cantidad de registros por {columna.name}: \n[{len(columna)}] 100%\n{(columna_frecuencia/columna_frecuencia.sum()*100).apply(lambda x: f"{x:.2f}%")}')    

In [12]:
MISSING_DATA_WAYS = ['', '-', 'Sin dato', 'Ilegible']
# Realizar regresion para el llenado de datos faltantes de cada columna
i = 0
for columna in data.columns:
	# print(f'Analizando columna {columna}')

	# Ver si un valor esta dentro de MISSING_DATA_WAYS
	if data[columna].isin(MISSING_DATA_WAYS).any():
		print(f'Columna {columna} tiene datos faltantes')

		# Cambiar los valores faltantes por NaN
		data[columna] = data[columna].replace(MISSING_DATA_WAYS, np.nan)

		# Ver si la columna es numerica excluyendo los datos faltantes
		try:
			if data[columna].dropna().str.isnumeric().all():
				print(f'Columna {columna} es numerica')
		except AttributeError:
			print(data[columna])
			pass

Columna TOTAL_BOLETAS_SOBRANTES tiene datos faltantes
Columna TOTAL_BOLETAS_SOBRANTES es numerica
Columna TOTAL_PERSONAS_VOTARON tiene datos faltantes
Columna TOTAL_PERSONAS_VOTARON es numerica
Columna TOTAL_REP_PARTIDO_CI_VOTARON tiene datos faltantes
Columna TOTAL_REP_PARTIDO_CI_VOTARON es numerica
Columna TOTAL_VOTOS_SACADOS tiene datos faltantes
Columna TOTAL_VOTOS_SACADOS es numerica
Columna PAN tiene datos faltantes
Columna PAN es numerica
Columna PRI tiene datos faltantes
Columna PRI es numerica
Columna PRD tiene datos faltantes
Columna PRD es numerica
Columna MC tiene datos faltantes
Columna MC es numerica
Columna PANAL tiene datos faltantes
Columna PANAL es numerica
Columna PVEM tiene datos faltantes
Columna PVEM es numerica
Columna PES tiene datos faltantes
Columna PES es numerica
Columna M tiene datos faltantes
Columna M es numerica
Columna PT tiene datos faltantes
Columna PT es numerica
Columna CQ tiene datos faltantes
Columna CQ es numerica
Columna QI tiene datos faltantes

In [13]:
resume_column(data[data['DISTRITO_LOCAL']=='QUERETARO 13']['UBICACION_CASILLA'])

Cantidad de registros por UBICACION_CASILLA: 
[193] 100%
NO URBANA    66.84%
URBANA       33.16%
Name: UBICACION_CASILLA, dtype: object


In [14]:
resume_column(data['PAN_PRD_MC'])


Cantidad de registros por PAN_PRD_MC: 
[2648] 100%
0     23.29%
1     18.63%
2     16.46%
3     14.91%
4     11.80%
5      7.45%
6      4.97%
7      1.86%
8      0.31%
10     0.31%
Name: PAN_PRD_MC, dtype: object


In [15]:
resume_column(data['UBICACION_CASILLA'])


Cantidad de registros por UBICACION_CASILLA: 
[2648] 100%
URBANA       55.59%
NO URBANA    44.41%
Name: UBICACION_CASILLA, dtype: object


In [16]:
resume_column(data[data[]])


SyntaxError: invalid syntax. Perhaps you forgot a comma? (3495289074.py, line 1)